In [1]:
import numpy as np
import pymongo
from bson.objectid import ObjectId
import requests
import json


## API pokemon

In [2]:
rayquaza_data = requests.get("https://pokeapi.co/api/v2/pokemon/rayquaza/").content
kyogre_data = requests.get("https://pokeapi.co/api/v2/pokemon/kyogre/").content
groudon_data = requests.get("https://pokeapi.co/api/v2/pokemon/groudon/").content
abra_data = requests.get("https://pokeapi.co/api/v2/pokemon/abra/").content


## Init connection 

In [3]:
client = pymongo.MongoClient("localhost", 27017)
pkm_db = client["pokemon"]

In [4]:
info = pkm_db["info"]

## Convert json format to dictionary in python

In [5]:
def convert_to_dict(bytes_str: bytes):
    temp = bytes_str.decode("utf-8")

    return json.loads(temp)

## Get pokemons

In [8]:
def load_all_pkm_data():
    prefix = "https://pokeapi.co/api/v2/pokemon"
    count = json.loads(requests.get(prefix).content)["count"]

    pkms = []

    for i in range(int(count/2)):
        pkm = convert_to_dict(requests.get(f"{prefix}/{i+1}").content)

        # print(pkm)
        pkms.append(pkm)

    return pkms

    
pkms = load_all_pkm_data()
len(pkms)

651

## Get pokemon's type

In [9]:
def get_types(pkm_data: dict):
    types = []

    for slot in pkm_data["types"]:
        types.append(slot["type"]["name"])

    return types

## Get pokemon's abilities

In [10]:
def get_abilities(pkm_data:dict):
    abilities = []

    for slot in pkm_data["abilities"]:
        abilities.append(slot["ability"]["name"])

    return abilities

## Get pokemon sprites

In [11]:
def get_sprites(pkm_data:dict):
    return pkm_data["sprites"]

## Push pokemon's info

In [12]:
for pkm in pkms:
    data_form = {
        "name":pkm["name"],
        "type":get_types(pkm),
        "abilities":get_abilities(pkm),
        "sprites":get_sprites(pkm)
        
    }

    pkm_db.info.insert_one(data_form)